In [55]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))
wight1=0
wight2=0

def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        # print('-----------')
        # print(equivalence_class)
        for key_decision in equivalence_classes_dec.keys():
            # print(key_decision)
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(x)
            if(len(x)>total_records*0):
                y = 1-len(x) / class_size
                # if y<0.7:
                total_rule+=len(x)
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)

    rule_confidences = np.array(rule_confidences)
    # print(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);


    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(count/total_records)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    random.seed(42)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof

In [56]:
def bestIndividual_KNN(hof,df):
    k = 2
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/an/anneal_train.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [57]:
from sklearn.model_selection import StratifiedKFold
def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=200)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('data/an/anneal_train.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [58]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('data/an/anneal_train.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier()
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [59]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,0.1,0.2,0.3,0.4,0.5]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=30
            n_gen=30
            # read dataframe from csv
            df = pd.read_csv('data/an/anneal_train.csv')
            y = df['target']
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            # for col in feature_columns:
            #     df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
            # get accuracy with all features
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")

            # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue

Accuracy with all features: 	(0.0,)

gen	nevals	avg      	min
0  	30    	0.0383399	0  
1  	21    	0.0133954	0  
2  	21    	0.00247989	0  
3  	24    	0.00158808	0  
4  	11    	0.000175439	0  
5  	18    	0          	0  
6  	22    	0.000429553	0  
7  	23    	0.0013376  	0  
8  	11    	0.00183666 	0  
9  	24    	0.000432001	0  
10 	25    	8.79507e-05	0  
11 	7     	0.00018018 	0  
12 	16    	0.000125   	0  
13 	17    	0.00118343 	0  
14 	20    	0.000438966	0  
15 	17    	0.0038904  	0  
16 	19    	0.0017318  	0  
17 	23    	0.000995025	0  
18 	15    	0.000520833	0  
19 	14    	8.726e-05  	0  
20 	23    	0.00054182 	0  
21 	15    	0.00143855 	0  
22 	17    	0.00015015 	0  
23 	20    	0.00180661 	0  
24 	20    	0.000166251	0  
25 	22    	0.00169269 	0  
26 	15    	0.000469484	0  
27 	14    	0.000165017	0  
28 	15    	0.00172083 	0  
29 	16    	0.000377929	0  
30 	20    	0.000133869	0  
Top 1 solution: [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 

In [60]:
print(SVM)
print(KNN)
print(DT)
print(F)

[[0.9766292134831461, 0.9933208489388266, 0.9966541822721598, 0.9966541822721598, 0.9955430711610488, 0.9966541822721598], [0.9766292134831461, 0.9153682896379527, 0.9944319600499376, 0.9153682896379527, 0.9175905118601749, 0.9966541822721598], [0.9766292134831461, 0.9153682896379527, 0.9175905118601749, 0.9187016229712859, 0.9187016229712859, 0.8853058676654182], [0.9031210986267165, 0.8841947565543071, 0.8307615480649189, 0.842983770287141, 0.8841822721598003, 0.8095755305867666], [0.8507615480649189, 0.8185018726591761, 0.8140699126092384, 0.7995505617977529, 0.7995505617977529, 0.7995505617977529], [0.7561298377028713, 0.7995505617977529, 0.8017727840199751, 0.8852933832709114, 0.8852933832709114, 0.8852933832709114]]
[[0.9566292134831462, 0.9922097378277155, 0.9933208489388266, 0.9933208489388266, 0.9899875156054933, 0.9922097378277155], [0.9566292134831462, 0.9142946317103622, 0.9933208489388266, 0.9142946317103622, 0.9142946317103622, 0.9944319600499376], [0.9566292134831462, 0.